In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas as gpd

from scipy.stats import binom
from sklearn.linear_model import LinearRegression

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 40)

In [43]:
# Collecte des données

data=pd.read_csv("Donnees-sur-le-parc-de-vehicule-au-niveau-regional.2023-05.csv", sep=';',header=[1])
zones = gpd.read_file('aires.geojson')

In [3]:
variables = ['PARC_2011','PARC_2012','PARC_2013','PARC_2014','PARC_2015','PARC_2016','PARC_2017','PARC_2018','PARC_2019','PARC_2020','PARC_2021','PARC_2022']
regions = data['REGION_LIBELLE'].unique()
regionsr = ['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne',
       'Centre-Val de Loire', 'Corse', 'Grand Est', 'Guadeloupe',
       'Guyane', 'Hauts-de-France', 'La Réunion',
       'Martinique', 'Mayotte', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
regionsmet = ['Auvergne-Rhône-Alpes', 'Bourgogne-Franche-Comté', 'Bretagne',
       'Centre-Val de Loire', 'Grand Est', 
        'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
regionsmethorsb = ['Auvergne-Rhône-Alpes', 'Bretagne',
       'Centre-Val de Loire', 'Grand Est', 
        'Hauts-de-France', 'Normandie', 'Nouvelle-Aquitaine',
       'Occitanie', 'Pays de la Loire', "Provence-Alpes-Côte d'Azur",
       'Île-de-France']
critair = data['CRITAIR'].unique()

In [6]:
data.sample(5)

,REGION_CODE,REGION_LIBELLE,CLASSE_VEHICULE,CATEGORIE_VEHICULE,CARBURANT,AGE,STATUT_UTILISATEUR,CRITAIR,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022
29802,52,Pays de la Loire,vul,Dérivé VP,Essence HNR,5 ans,Professionnel,Crit'Air 1,0,0,0,0,0,0,1,0,2,3,1,2
21136,04,La Réunion,vul,Camionnette,Gaz et inconnu,8 ans,Professionnel,Non classé,1,0,0,0,0,0,0,0,0,0,0,0
32584,11,Île-de-France,vul,Dérivé VP,Diesel,15 ans,Professionnel,Crit'Air 3,0,0,0,0,0,0,0,0,0,0,3,540
8263,27,Bourgogne-Franche-Comté,vul,Camping-car,Diesel,15 ans,Particulier,Crit'Air 3,0,0,0,0,0,0,0,0,0,0,1,371
48086,11,Île-de-France,vp,Véhicule particulier,Gaz et inconnu,3 ans,Professionnel,Crit'Air 2,3,6,2,1,1,1,0,0,0,0,0,0


In [59]:
data_fil = data.loc[(data["CATEGORIE_VEHICULE"] == "Véhicule particulier") | (data["CATEGORIE_VEHICULE"] == "Dérivé VP") | (data["CATEGORIE_VEHICULE"] == "Camionnette")]
data_fil['Pannes'] = 0
data_fil.groupby(['REGION_CODE','CATEGORIE_VEHICULE','CARBURANT','AGE'])

for i in range (2) :
    pannes_annees = []
    for j in range (11, 22) :
        var = "PARC_20" + str(j)
        var_2 = "PARC_20" + str(j + 1)
        age = int(data_fil.iloc[i]["AGE"][0:2])
        if age == 0 :
            age_plus_1 = "1 an"
        elif age == 24 :
            age_plus_1 = "25 ans et plus"
        else : 
            age_plus_1 = str(age + 1) + " ans"
        data_plus_1 = data_fil.loc[
                        (data_fil["REGION_CODE"] == data_fil.iloc[i]["REGION_CODE"]) &
                        (data_fil["CATEGORIE_VEHICULE"] == data_fil.iloc[i]["CATEGORIE_VEHICULE"]) &
                        (data_fil["CARBURANT"] == data_fil.iloc[i]["CARBURANT"]) &
                        (data_fil["CRITAIR"] == data_fil.iloc[i]["CRITAIR"]) &
                        (data_fil["STATUT_UTILISATEUR"] == data_fil.iloc[i]["STATUT_UTILISATEUR"]) &
                        (data_fil["AGE"] == age_plus_1)
        ]
        pannes_annees.append(data_fil.iloc[i][var] - data_plus_1.iloc[0][var_2])
    data_fil2 = data_fil.copy()
    data_fil2.iloc[i]['Pannes'] = np.mean(pannes_annees)
    print(np.mean(pannes_annees))
    print(data_fil2.iloc[i]['Pannes'])



/tmp/ipykernel_397/3745587453.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fil['Pannes'] = 0
/tmp/ipykernel_397/3745587453.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_fil2.iloc[i]['Pannes'] = np.mean(pannes_annees)


291.09090909090907
0
43.0
0


In [57]:
data_fil.head(5)

,REGION_CODE,REGION_LIBELLE,CLASSE_VEHICULE,CATEGORIE_VEHICULE,CARBURANT,AGE,STATUT_UTILISATEUR,CRITAIR,PARC_2011,PARC_2012,PARC_2013,PARC_2014,PARC_2015,PARC_2016,PARC_2017,PARC_2018,PARC_2019,PARC_2020,PARC_2021,PARC_2022,Pannes
5814,84,Auvergne-Rhône-Alpes,vul,Camionnette,Diesel,0 an,Professionnel,Crit'Air 2,2069,15690,29936,30528,30441,31624,34236,39301,39921,40774,35327,38256,0
5816,84,Auvergne-Rhône-Alpes,vul,Dérivé VP,Diesel,0 an,Professionnel,Crit'Air 2,438,10945,10272,9690,10103,10977,12236,12254,11657,10149,7460,6032,0
5818,84,Auvergne-Rhône-Alpes,vul,Camionnette,Diesel,0 an,Professionnel,Crit'Air 3,31772,19236,2723,36,2,0,1,0,0,0,0,0,0
5820,84,Auvergne-Rhône-Alpes,vul,Dérivé VP,Diesel,0 an,Professionnel,Crit'Air 3,9874,65,3,0,1,0,0,0,0,0,0,0,0
5822,84,Auvergne-Rhône-Alpes,vul,Camionnette,Diesel,0 an,Professionnel,Crit'Air 4,0,3,0,0,0,0,0,0,0,0,0,0,0
